In [1]:
import wget
import ssl
import os
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# Enter NCRB Individual PDF url
pdf_url = 'https://ncrb.gov.in/sites/default/files/adsi_reports_previous_year/Table-1A.7_2019.pdf'
fileName = pdf_url[67:]
try:
    #Delete the file to redownload it
    os.remove(fileName)
except FileNotFoundError as e:
    #Ignore in case FileNotFound Error occurs
    pass
# Download report
fileName = wget.download(pdf_url,fileName)

In [3]:
import numpy as np
import pandas as pd
import camelot

In [4]:
fileName

'Table-1A.7_2019.pdf'

In [5]:
table1_2 = camelot.read_pdf(fileName,
                          flavor='stream',
                          table_areas=[
                              "33.18688206785138,728.5539356601939,565.1389337641358,72.4997530453068"
                              ],
                          columns=[
                              "52,157.75793214862682,203.93098546042006,246.25628432956384,284.7338287560582,331.86882067851377,373.2321809369952,423.25298869143785,477.12155088852995,520.4087883683361"
                              ],
                          split_text=True,
                          pages="1-2"
                         )

In [6]:
table1_2[1].df.head(15)

,0,1,2,3,4,5,6,7,8,9,10
0,(1),(2),(3),(4),(5),(6),(7),(8),(9),(10),(11)
1,37,AGRA,36,10,44,85,15,97,23,6,24
2,38,AHMEDABAD,136,71,69,76,49,26,6,3,3
3,39,ALLAHABAD,197,146,82,166,130,77,0,0,0
4,40,AMRITSAR,30,19,21,48,28,31,0,0,0
5,41,ASANSOL,205,164,146,34,26,24,0,0,0
6,42,AURANGABAD,30,26,17,116,67,68,0,0,0
7,43,BENGALURU,761,770,167,211,188,40,0,0,0
8,44,BHOPAL,206,152,18,985,657,61,0,0,0
9,45,CHANDIGARH (CITY),36,27,13,0,0,0,0,0,0


In [7]:
# Fix Coimbatore's mis-read by Camelot
table1_2[1].df.iloc[11,0] = table1_2[1].df.iloc[12,2]
table1_2[1].df.iloc[11,1] = table1_2[1].df.iloc[12,1]
# Drop the faulty Row
table1_2[1].df.drop(index = 12,inplace=True)
# And Reset the index
table1_2[1].df.index = range(len(table1_2[1].df))

In [8]:
table3_4 = camelot.read_pdf(fileName,
                          flavor='stream',
                          table_areas=[
                              "49.53983844911148,729.4838345587917,564.1769951534734,71.50573938491661"
                              ],
                          columns=[
                              "69.25957996768983,190.46384491114702,233.75108239095317,298.2009693053312,366.49861066235866,429.024620355412,501.1700161550889"
                              ],
                          split_text=True,
                          pages="3-4"
                         )

In [9]:
# tables

In [10]:
def printnAllTables(tables):
    for i in range(len(tables)):
        print("Page No: ",(i+1))
        print(tables[i].df)
        print()

In [11]:
# Incase we're intersted in seeing the original scraped tables.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

In [12]:
# Removes Garbage Rows that may have been detected Before the 'STATES' row
# AND
# Removes Garbage Rows that may have been detected after the 'TOTAL (ALL INDIA)' row
def clean(table):
    while(table.df.iloc[0,1] != 'STATES' and table.df.iloc[0,1] != 'AGRA'):
        table.df = table.df.iloc[1:]
    while(table.df.iloc[-1,1] != 'TOTAL (ALL INDIA)' and table.df.iloc[-1,1] != 'TOTAL (CITIES)'):
        table.df = table.df.iloc[:-1]
    # The Return line is unnecessary due to all changes being reflected in the original references
    return table

In [13]:
# Splits the second column into 1st column index and 2nd column test when faulty/required
def split_by_first_space(df, strIndex, stpIndex):
    df = df.iloc[strIndex:stpIndex+1]
    for index, each in df.iterrows():
        space = each[1].find(' ')
        each[0] = each[1][:space]
        each[1] = each[1][space:]

In [14]:
def mergeRows(df, strIndex, stpIndex):
    DF = df.loc[strIndex:stpIndex]
    df = DF.T
    l = []
    for index, each in df.iterrows():
        output = ''
        for ind, cell in each.iteritems():
            output += (str(cell) + ' ')
#         if(output == ''):
#             continue
        l.append(output.strip())
    l = pd.Series(l,name=str(strIndex))
    DF.loc[strIndex] = l

In [15]:
for each_table in table1_2:
    clean(each_table)

In [16]:
for each_table in table3_4:
    clean(each_table)

In [17]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(table1_2)
# printnAllTables(table3_4)

In [18]:
states = [table1_2[0].df.copy(), table3_4[0].df.copy()]
cities = [table1_2[1].df.copy(), table3_4[1].df.copy()]

In [19]:
# List of Column headers as column are nested, i.e exhibit a multi level/index structure
# The multi level structre will be generated and applied in the next 
column_header = [
    ['National Highways','State Highways','Expressways'],
    ['Other Roads','Total']
]
len(column_header)

2

In [20]:
def set_Column(df, title, state, last):
    if(not last):
        column_list = [
            ("Sl. No.",''),
            ("State/UT" if(state) else "City",''), 
            (title[0],'Cases'),
            (title[0],'Injured'),
            (title[0],'Dead'),
            (title[1],'Cases'),
            (title[1],'Injured'),
            (title[1],'Dead'),
            (title[2],'Cases'),
            (title[2],'Injured'),
            (title[2],'Dead')
            ]
    else:
        column_list = [
            ("Sl. No.",''),
            ("State/UT" if(state) else "City",''), 
            (title[0],'Cases'),
            (title[0],'Injured'),
            (title[0],'Dead'),
            (title[1],'Cases'),
            (title[1],'Injured'),
            (title[1],'Dead')
            ]
    df.columns = pd.MultiIndex.from_tuples(column_list)

In [21]:
for i,(state,city) in enumerate(zip(states,cities)):
    set_Column(df=state,title=column_header[i],state=True,last=i)
    state.set_index(["Sl. No.","State/UT"],inplace=True)
    set_Column(df=city,title=column_header[i],state=False,last=i)
    city.set_index(["Sl. No.","City"],inplace=True)

In [22]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

# OR Print a simgle one according to your choice
# tables[23].df

In [23]:
# Concat all the tables one after the other, as they all have the same indexes.
State = pd.concat(states,axis=1)
City  = pd.concat(cities,axis=1)

In [24]:
# Reset Index before saving file, for better formatting in RAW CSV
State.reset_index().to_csv("Road Classification – wise Number of Road Accidents, Injuries and Deaths during 2019(StateUT-wise).csv",index=False)
City.reset_index().to_csv("Road Classification – wise Number of Road Accidents, Injuries and Deaths during 2019(City-wise).csv",index=False)